# 测试 TVM 结构

{func}`tvm.ir.base.structural_equal` 在 IRNodes 的 DAG 中递归地定义结构相等。

节点有两种：

- Graph 节点：lhs 中的 graph 节点只能映射为等于 rhs 中的且只能映射为一个 graph 节点。
- 普通节点：相等是递归定义的，不受 graph 节点的限制。

Vars(tir::Var, TypeVar) 和不是常量的 Relay 表达式节点是 Graph 节点。例如，在 Relay 中 `%1 = %x + %y; %1 + %1` 在结构上不等于 `%1 = %x + %y; %2 = %x + %y; %1 + %2`。

如果下列条件之一成立，var-type 节点（如 tir::Var, TypeVar）可以被映射为与另一个相同类型的 Var 相等：

- 它们出现在同一个定义点（例如：函数参数）。
- 它们通过 same_as 关系指向相同的 VarNode。
- 它们出现在同一个 usage 点，并且 map_free_vars 设置为 `True`。

var 规则用于重新映射函数参数和 `let` 绑定中的变量。

In [1]:
import numpy as np
import tvm
import tvm.testing
from tvm import relay
from tvm.relay.testing import run_opt_pass

In [3]:
import pytest
import numpy as np

import tvm
from tvm.runtime import vm as _vm
from tvm.relay import vm as rly_vm
from tvm import relay

from tvm.relay.scope_builder import ScopeBuilder
from tvm.relay import transform
from tvm.relay.prelude import Prelude
from tvm.contrib import utils
from tvm.relay import testing